In [1]:
import numpy as np
import pandas as pd
import time

In [ ]:
#cite: https://www.kaggle.com/selfishgene/nothing-fancy-just-some-heuristics-0-9374

In [3]:
#%% load data
print('loading data...')

farmerPref = pd.read_csv('Farmer_preferences.csv', header=None).as_matrix()[:, 1:]
buyerPref = pd.read_csv('Buyer_preferences.csv' , header=None).as_matrix()[:, 1:]

numBuyers = buyerPref.shape[0] #child is buyer
numFarmers = farmerPref.shape[0] #farmer is gift/santa
numFarmersPerBuyer = numBuyers / numFarmers #HEY LEANNA watch out - not every farmer is going to get matched

loading data...


/Users/leannamulvihill/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/Users/leannamulvihill/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [18]:
#%% create lookup matrix
print('creating buyer vs. farmer value matrix...')
buyerPreferenceMatrix = -1*np.ones((numBuyers,numFarmers),np.float32)
for buyerID in range(numBuyers):
    for farmerOrder, farmID in enumerate(buyerPref[buyerID,:]):
        #buyerPreferenceMatrix[buyerID,farmID] = 2*(15 - farmerOrder) 
        buyerPreferenceMatrix[buyerID,farmID] = (15 - farmerOrder)
        
farmerPreferenceMatrix = -1*np.ones((numBuyers,numFarmers),np.float32)
for farmID in range(numFarmers):
    for buyerOrder, buyerID in enumerate(farmerPref[farmID,:]):
        #farmerPreferenceMatrix[buyerID,farmID] = 2*(5 - buyerOrder)
        farmerPreferenceMatrix[buyerID,farmID] = (5 - buyerOrder)

#buyer_vs_farmer_matrix =  buyerPreferenceMatrix/(20.0*numBuyers)
buyer_vs_farmer_matrix =  buyerPreferenceMatrix/(numBuyers)
#buyer_vs_farmer_matrix += farmerPreferenceMatrix/(2000000.0*numFarmers)
buyer_vs_farmer_matrix += farmerPreferenceMatrix/(numFarmers)
buyer_vs_farmer_matrix = buyer_vs_farmer_matrix.astype(np.float32)

# del buyerPreferenceMatrix
# del farmerPreferenceMatrix

creating buyer vs. farmer value matrix...


In [19]:
# print(buyerPreferenceMatrix) #this looks good
# print(farmerPreferenceMatrix) #does it make sense that this matrix is the same size as buyerPreferenceMatrix? Yes.
# print(buyer_vs_farmer_matrix.shape) #looks good

In [21]:
# scoring function
def calculateTotalHappiness(pred):
    totalHappiness = 0

    for i in range(numBuyers):
        buyer_id = i
        farmer_id = pred[i]
        totalHappiness += buyer_vs_farmer_matrix[buyer_id,farmer_id]
        
    return totalHappiness

In [29]:
def AssignFarmers_GreedyBuyers_Adaptive(buyer_vs_farmer_selection_matrix=buyer_vs_farmer_matrix, numPasses=15):
    buyer_vs_farmer_selection_matrix = buyer_vs_farmer_selection_matrix.copy()
    
    farmerAssignment = -np.ones((numBuyers), dtype=np.int32)
    farmerCount      = np.zeros((numFarmers),    dtype=np.int32)
    
    print('-'*40)
    startTime  = time.time()
            
    print('starting adaptive pass over the buyers')
    buyersPerPass = int(1+(numBuyers) / (numPasses+1.0))
    
    # at each pass we lower a threshold and assign gift to childern that are above the threshold
    for k in range(numPasses+1):
        # sort the children accroding to the maximum possible matrix value of each child
        maxValuePerBuyer = buyer_vs_farmer_selection_matrix.max(axis=1)
        sortedBuyers   = (maxValuePerBuyer.argsort())[::-1]
        
        thresholdBuyerInd   = numBuyers-1        
        assignmentThreshold = maxValuePerBuyer[thresholdBuyerInd]

        numAssignedSoFar = (farmerAssignment > -1).sum()        
        if (numAssignedSoFar > (0.99*numBuyers)) or (k >= (numPasses)):
            # make this last iteration
            assignmentThreshold = buyer_vs_farmer_selection_matrix.min() - 1.0
            thresholdBuyerInd = len(sortedBuyers)
            
        if numAssignedSoFar >= numBuyers:
            break
        
        for buyerInd in sortedBuyers[:thresholdBuyerInd]:
            # don't assign gifts unless high on priority list ( larger than 'assignmentThreshold' )
            if farmerAssignment[buyerInd] == -1 and buyer_vs_farmer_selection_matrix[buyerInd,:].max() >= assignmentThreshold:
                selectedFarmer = buyer_vs_farmer_selection_matrix[buyerInd,:].argmax()
        
                farmerAssignment[buyerInd] = selectedFarmer
                farmerCount[selectedFarmer] += 1
                
                if farmerCount[selectedFarmer] >= numFarmersPerBuyer:
                    buyer_vs_farmer_selection_matrix[:,selectedFarmer] = -1.0
                    
        print('pass %d: total assigned so far = %d' %(k+1,(farmerAssignment > -1).sum()))

    print('finished %d adaptive passes. took %.3f seconds' %(k+1, time.time()-startTime))
    assignmentScore = calculateTotalHappiness(farmerAssignment)
    print('-'*40)

    return farmerAssignment, assignmentScore

In [30]:
#%% normalize rows and columns of the assignment value matrix
print('normalizing rows and columns of the value matrix')
startingMatrix = buyer_vs_farmer_matrix.copy()
startingMatrix -= startingMatrix.min()
startingMatrix /= startingMatrix.max()

normalizing rows and columns of the value matrix


In [32]:
# create an assignment vector from the child vs. gift value matrix
print('assign farmers to buyers in a buyer centered greedy fashion')
pred, score = AssignFarmers_GreedyBuyers_Adaptive(startingMatrix, numPasses=16)
print('predicted score = %.8f' %(score))

assign farmers to buyers in a buyer centered greedy fashion
----------------------------------------
starting adaptive pass over the buyers
pass 1: total assigned so far = 1
pass 2: total assigned so far = 2
pass 3: total assigned so far = 3
pass 4: total assigned so far = 4
pass 5: total assigned so far = 5
finished 6 adaptive passes. took 0.002 seconds
----------------------------------------
predicted score = 14.66666627


In [33]:
#%% create a submission
out = open('heuristicSub.csv', 'w')
out.write('buyerId,farmId\n')
for i in range(len(pred)):
    out.write(str(i) + ',' + str(pred[i]) + '\n')
out.close()